In [1]:
load('../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [3]:
# Define cubic SE4, find and classify the lines and compute the 45 tritangent planes

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE4 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)})

In [5]:
# Find simmetries

In [6]:
%time adm_SE4 = SE4.find_admissible_projectivities()
len(adm_SE4)

CPU times: user 448 ms, sys: 52.5 ms, total: 501 ms
Wall time: 1.07 s


36

In [7]:
%time simm_SE4 = SE4.find_simmetries(adm_SE4)
len(simm_SE4)

CPU times: user 203 ms, sys: 64.5 ms, total: 268 ms
Wall time: 385 ms


12

In [8]:
# Study how the simmetries permute the Eckardt points

In [9]:
Eck_perms = [Permutation(SE4.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE4]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(6, 'S3')

In [10]:
# Study how the simmetries permute the 27 lines

In [11]:
lines_perms = [Permutation(SE4.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE4]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(12, 'D6')

In [12]:
from_labels_to_perm(from_perm_to_labels(lines_perms[0]))

(1,9)(2,10)(3,7)(4,8)(5,12)(6,11)(13,22)(15,18)(16,24)(17,23)(20,26)(21,25)

In [13]:
# D6 is isomorphic to C2 x S3

In [14]:
nor_subs = lines_perms_group.normal_subgroups()
index_S3 = [el.is_isomorphic(SymmetricGroup(3)) for el in nor_subs].index(True)
S3 = nor_subs[index_S3]
index_C2 = [el.is_isomorphic(lines_perms_group.quotient(S3)) for el in nor_subs].index(True)
C2 = nor_subs[index_C2]

In [15]:
[get_permuted_extended_L_set(perm) for perm in S3.gens_small()+C2.gens_small()],

([('G3', 'E2', 'F24', 'F13', 'F12', 'F26'),
  ('E1', 'G4', 'F24', 'F13', 'F34', 'F45'),
  ('F14', 'G4', 'E2', 'F23', 'F34', 'F45')],)

In [16]:
lines_perms_group.is_isomorphic(direct_product_permgroups([S3,C2]))

True

In [18]:
# S3 acts as the symmetric group on A, C, F, while fixing D.
# C2 fixes the four Eckardt points, permutes lines not passing through D
keys = list(general_cubic.cl_lines.keys())
for gen in S3.gens_small()+C2.gens_small():
    labels = from_perm_to_labels(gen)
    print([[keys[labels.index(label)] for label in point_label] for point_label in SE4.eckardt_points_labels])

[['F13', 'F24', 'F56'], ['G4', 'E1', 'F14'], ['G4', 'F24', 'E2'], ['G3', 'E2', 'F23']]
[['E1', 'G4', 'F14'], ['F24', 'F13', 'F56'], ['F24', 'G4', 'E2'], ['G3', 'E2', 'F23']]
[['F14', 'G4', 'E1'], ['E2', 'F23', 'G3'], ['E2', 'G4', 'F24'], ['F56', 'F24', 'F13']]


In [19]:
print(SE4.eckardt_points_labels)

[['E1', 'G4', 'F14'], ['E2', 'G3', 'F23'], ['E2', 'G4', 'F24'], ['F13', 'F24', 'F56']]


In [20]:
# Find possible conditions to obtain a subfamily with a larger simmetry group

In [21]:
%time SE4.find_conditions_for_subfamilies(adm_SE4, simm_SE4)

CPU times: user 116 ms, sys: 72.5 ms, total: 188 ms
Wall time: 518 ms


[]